In [5]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [7]:
subject = 'Rio Grande Do Norte - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [8]:
data = pd.read_csv('2003_mo_model_input_RN.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Rio Grande do Norte - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande Do Norte - Produção de Cimento (t),Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande do Norte - Desemprego,Rio Grande Do Norte - Consumo de Cimento (t)
0,2003-1,0.689488,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,27.698760,3.164847e+07,1.998664e+06,9.097985,2.879532e+07,8.294170,36.825
1,2003-2,0.689892,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,27.407645,3.168897e+07,2.001107e+06,9.101271,2.881033e+07,8.288224,34.088
2,2003-3,0.690297,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,27.377422,3.172947e+07,2.003550e+06,9.104557,2.882535e+07,8.282278,30.150
3,2003-4,0.690702,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,27.542280,3.176997e+07,2.005994e+06,9.107843,2.884036e+07,8.276332,31.224
4,2003-5,0.691107,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,27.944900,3.181046e+07,2.008437e+06,9.111128,2.885538e+07,8.270387,33.606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.557326,NaN,NaN,NaN,NaN,NaN,83.890
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.910014,NaN,NaN,NaN,NaN,NaN,83.438
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.800783,NaN,NaN,NaN,NaN,NaN,80.005
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.752983,NaN,NaN,NaN,NaN,NaN,75.441


In [9]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Rio Grande do Norte - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande Do Norte - Produção de Cimento (t),Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande do Norte - Desemprego
0,-2.184058,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.183161,-1.731345,-0.778207,-2.784513,-2.193066,-0.876481
1,-2.147573,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.192364,-1.711183,-0.728728,-2.703361,-2.146603,-0.878896
2,-2.111089,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.193319,-1.691021,-0.679248,-2.622210,-2.100140,-0.881311
3,-2.074605,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.188108,-1.670859,-0.629768,-2.541059,-2.053677,-0.883727
4,-2.038120,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.175380,-1.650697,-0.580289,-2.459908,-2.007214,-0.886142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.321156,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.389536,1.102751,-1.573216,0.457957,0.775635,1.052347
188,1.314269,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,0.398528,1.092237,-1.561825,0.427807,0.759230,1.051753
189,1.307381,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,0.414978,1.081724,-1.550435,0.397658,0.742826,1.051160
190,1.300494,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,0.429834,1.071210,-1.539044,0.367509,0.726421,1.050566


In [10]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      32.845
1      26.809
2      34.465
3      30.563
4      32.574
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Rio Grande Do Norte - Consumo de Cimento (t), Length: 240, dtype: float64

In [11]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,Rio Grande do Norte - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande Do Norte - Produção de Cimento (t),Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande do Norte - Desemprego
0,-2.184058,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.183161,-1.731345,-0.778207,-2.784513,-2.193066,-0.876481
1,-2.147573,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.192364,-1.711183,-0.728728,-2.703361,-2.146603,-0.878896
2,-2.111089,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.193319,-1.691021,-0.679248,-2.622210,-2.100140,-0.881311
3,-2.074605,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.188108,-1.670859,-0.629768,-2.541059,-2.053677,-0.883727
4,-2.038120,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.175380,-1.650697,-0.580289,-2.459908,-2.007214,-0.886142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.384282,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.046207,1.183768,-1.442416,0.625176,0.985411,1.213189
158,1.382302,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.000373,1.187707,-1.461501,0.642268,0.990347,1.201976
159,1.380323,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,0.957199,1.191647,-1.480587,0.659360,0.995283,1.190764
160,1.378343,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,0.908410,1.195587,-1.499673,0.676453,1.000219,1.179551


In [12]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0      32.845
1      26.809
2      34.465
3      30.563
4      32.574
        ...  
157    52.722
158    63.109
159    49.356
160    55.725
161    50.796
Name: Rio Grande Do Norte - Consumo de Cimento (t), Length: 162, dtype: float64

In [13]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Rio Grande do Norte - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande Do Norte - Produção de Cimento (t),Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande do Norte - Desemprego
162,1.374384,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,0.848293,1.203467,-1.537845,0.710637,1.010091,1.157126
163,1.372404,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,0.824099,1.207407,-1.556931,0.727729,1.015027,1.145914
164,1.370425,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,0.799099,1.211347,-1.576017,0.744821,1.019963,1.134701
165,1.368445,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,0.777768,1.215287,-1.595102,0.761913,1.024899,1.123489
166,1.366466,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,0.760884,1.219226,-1.614188,0.779005,1.029835,1.112277
167,1.364486,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,0.735834,1.223166,-1.633274,0.796097,1.034770,1.101064
168,1.362506,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,0.707114,1.227106,-1.652360,0.813189,1.039706,1.089852
169,1.363078,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,0.679884,1.222876,-1.652409,0.801173,1.027270,1.087072
170,1.363650,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,0.645700,1.218646,-1.652458,0.789158,1.014833,1.084293
171,1.364221,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,0.600550,1.214416,-1.652508,0.777142,1.002396,1.081514


In [14]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162    48.750
163    59.738
164    60.303
165    51.009
166    48.874
167    48.506
168    42.721
169    32.801
170    37.638
171    37.358
172    31.339
173    45.311
174    42.085
175    48.520
176    41.365
177    43.813
178    43.816
179    38.245
180    46.081
181    38.703
182    36.037
183    38.430
184    44.471
185    37.884
186    51.585
187    54.306
188    52.511
189    59.472
190    54.022
191    49.791
Name: Rio Grande Do Norte - Consumo de Cimento (t), dtype: float64

In [15]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*div_factor + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [19]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 8)
    target,target_val = validation_splitter(train_target, 8)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [20]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [21]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1915252632, 3219293133, 3285064327, 462069691, 2282234682, 418808632, 935154646, 184270058, 553343776, 2964421840, 4233998320, 1966993840, 2441000978, 2076917689, 3099940549, 103510955, 1425340031, 2539920630, 240577878, 1546965392, 1316517200, 2328839478, 1280919727, 315834237, 911295073]


Step: 0 ___________________________________________
val_loss: 20.079696655273438
winner_seed: 1915252632


Step: 1 ___________________________________________
val_loss: 20.630374908447266


Step: 2 ___________________________________________
val_loss: 18.650821685791016
winner_seed: 3285064327


Step: 3 ___________________________________________
val_loss: 15.470341682434082
winner_seed: 462069691


Step: 4 ___________________________________________
val_loss: 15.458224296569824
winner_seed: 2282234682


Step: 5 ___________________________________________
val_loss: 15.547518730163574


Step: 6 ___________________________________________
val_loss: 16.271448135375977


Step: 7 ______________________

In [22]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 0s 20ms/step - loss: 3733.3223 - val_loss: 3518.5591
Epoch 2/10000
5/5 [==============================] - 0s 4ms/step - loss: 3102.3816 - val_loss: 2566.7329
Epoch 3/10000
5/5 [==============================] - 0s 4ms/step - loss: 2870.5706 - val_loss: 472.2202
Epoch 4/10000
5/5 [==============================] - 0s 4ms/step - loss: 2395.6458 - val_loss: 164.3064
Epoch 5/10000
5/5 [==============================] - 0s 4ms/step - loss: 194.8172 - val_loss: 75.0036
Epoch 6/10000
5/5 [==============================] - 0s 4ms/step - loss: 71.7318 - val_loss: 47.8899
Epoch 7/10000
5/5 [==============================] - 0s 4ms/step - loss: 53.6750 - val_loss: 49.0841
Epoch 8/10000
5/5 [==============================] - 0s 4ms/step - loss: 47.4989 - val_loss: 88.9597
Epoch 9/10000
5/5 [==============================] - 0s 4ms/step - loss: 56.9861 - val_loss: 45.0438
Epoch 10/10000
5/5 [==============================] - 0s 4ms/step - loss: 5

5/5 [==============================] - 0s 4ms/step - loss: 30.9553 - val_loss: 37.9672
Epoch 82/10000
5/5 [==============================] - 0s 4ms/step - loss: 30.0897 - val_loss: 32.8980
Epoch 83/10000
5/5 [==============================] - 0s 4ms/step - loss: 31.9472 - val_loss: 41.2586
Epoch 84/10000
5/5 [==============================] - 0s 5ms/step - loss: 32.5076 - val_loss: 29.7438
Epoch 85/10000
5/5 [==============================] - 0s 4ms/step - loss: 29.6915 - val_loss: 39.3659
Epoch 86/10000
5/5 [==============================] - 0s 4ms/step - loss: 27.7912 - val_loss: 26.7785
Epoch 87/10000
5/5 [==============================] - 0s 4ms/step - loss: 28.0115 - val_loss: 57.9322
Epoch 88/10000
5/5 [==============================] - 0s 4ms/step - loss: 34.4597 - val_loss: 29.4391
Epoch 89/10000
5/5 [==============================] - 0s 4ms/step - loss: 34.4791 - val_loss: 47.8388
Epoch 90/10000
5/5 [==============================] - 0s 4ms/step - loss: 33.2040 - val_loss: 25.

Epoch 161/10000
5/5 [==============================] - 0s 4ms/step - loss: 28.7627 - val_loss: 43.1920
Epoch 162/10000
5/5 [==============================] - 0s 4ms/step - loss: 23.5027 - val_loss: 34.7795
Epoch 163/10000
5/5 [==============================] - 0s 4ms/step - loss: 27.7532 - val_loss: 35.9709
Epoch 164/10000
5/5 [==============================] - 0s 4ms/step - loss: 29.7314 - val_loss: 48.2101
Epoch 165/10000
5/5 [==============================] - 0s 4ms/step - loss: 32.9259 - val_loss: 37.5144
Epoch 166/10000
5/5 [==============================] - 0s 4ms/step - loss: 28.2027 - val_loss: 36.4754
Epoch 167/10000
5/5 [==============================] - 0s 4ms/step - loss: 27.6799 - val_loss: 42.0627
Epoch 168/10000
5/5 [==============================] - 0s 5ms/step - loss: 31.0715 - val_loss: 33.3981
Epoch 169/10000
5/5 [==============================] - 0s 4ms/step - loss: 29.2359 - val_loss: 32.1019
Epoch 170/10000
5/5 [==============================] - 0s 4ms/step - loss

5/5 [==============================] - 0s 4ms/step - loss: 25.8746 - val_loss: 46.0185
Epoch 241/10000
5/5 [==============================] - 0s 4ms/step - loss: 24.5821 - val_loss: 39.1435
Epoch 242/10000
5/5 [==============================] - 0s 4ms/step - loss: 20.5417 - val_loss: 27.0233
Epoch 243/10000
5/5 [==============================] - 0s 5ms/step - loss: 22.2347 - val_loss: 25.2009
Epoch 244/10000
5/5 [==============================] - 0s 5ms/step - loss: 23.7894 - val_loss: 36.1856
Epoch 245/10000
5/5 [==============================] - 0s 5ms/step - loss: 21.9562 - val_loss: 39.9470
Epoch 246/10000
5/5 [==============================] - 0s 4ms/step - loss: 24.6973 - val_loss: 42.3468
Epoch 247/10000
5/5 [==============================] - 0s 5ms/step - loss: 32.2315 - val_loss: 26.2950
Epoch 248/10000
5/5 [==============================] - 0s 5ms/step - loss: 31.7468 - val_loss: 31.5635
Epoch 249/10000
5/5 [==============================] - 0s 4ms/step - loss: 26.8926 - val_

5/5 [==============================] - 0s 4ms/step - loss: 22.7850 - val_loss: 22.1702
Epoch 320/10000
5/5 [==============================] - 0s 4ms/step - loss: 21.6263 - val_loss: 23.6501
Epoch 321/10000
5/5 [==============================] - 0s 4ms/step - loss: 19.6974 - val_loss: 34.2867
Epoch 322/10000
5/5 [==============================] - 0s 4ms/step - loss: 21.4719 - val_loss: 31.3273
Epoch 323/10000
5/5 [==============================] - 0s 5ms/step - loss: 21.2425 - val_loss: 24.8050
Epoch 324/10000
5/5 [==============================] - 0s 4ms/step - loss: 22.4888 - val_loss: 36.3215
Epoch 325/10000
5/5 [==============================] - 0s 4ms/step - loss: 22.8529 - val_loss: 34.8729
Epoch 326/10000
5/5 [==============================] - 0s 4ms/step - loss: 22.0976 - val_loss: 31.4468
Epoch 327/10000
5/5 [==============================] - 0s 4ms/step - loss: 20.5414 - val_loss: 25.0284
Epoch 328/10000
5/5 [==============================] - 0s 4ms/step - loss: 20.5853 - val_

...

5/5 [==============================] - 0s 4ms/step - loss: 15.3203 - val_loss: 17.8902
Epoch 1425/10000
5/5 [==============================] - 0s 4ms/step - loss: 17.0191 - val_loss: 23.1287
Epoch 1426/10000
5/5 [==============================] - 0s 5ms/step - loss: 16.6427 - val_loss: 29.0706
Epoch 1427/10000
5/5 [==============================] - 0s 4ms/step - loss: 14.5213 - val_loss: 28.6476
Epoch 1428/10000
5/5 [==============================] - 0s 4ms/step - loss: 15.6109 - val_loss: 18.9977
Epoch 1429/10000
5/5 [==============================] - 0s 4ms/step - loss: 14.7255 - val_loss: 24.2008
Epoch 1430/10000
5/5 [==============================] - 0s 4ms/step - loss: 15.7974 - val_loss: 20.0289
Epoch 1431/10000
5/5 [==============================] - 0s 5ms/step - loss: 14.3348 - val_loss: 17.1385
Epoch 1432/10000
5/5 [==============================] - 0s 4ms/step - loss: 15.2217 - val_loss: 30.1191
Epoch 1433/10000
5/5 [==============================] - 0s 4ms/step - loss: 15.01

Epoch 1503/10000
5/5 [==============================] - 0s 5ms/step - loss: 16.5423 - val_loss: 29.4313
Epoch 1504/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.6200 - val_loss: 25.4312
Epoch 1505/10000
5/5 [==============================] - 0s 5ms/step - loss: 12.2019 - val_loss: 19.6020
Epoch 1506/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.4947 - val_loss: 16.3695
Epoch 1507/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.5076 - val_loss: 19.9065
Epoch 1508/10000
5/5 [==============================] - 0s 4ms/step - loss: 14.6215 - val_loss: 25.2238
Epoch 1509/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.3673 - val_loss: 17.0450
Epoch 1510/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.6897 - val_loss: 44.9311
Epoch 1511/10000
5/5 [==============================] - 0s 4ms/step - loss: 14.6002 - val_loss: 33.7650
Epoch 1512/10000
5/5 [==============================] - 0s 4ms/s

Epoch 1582/10000
5/5 [==============================] - 0s 4ms/step - loss: 15.2007 - val_loss: 21.7515
Epoch 1583/10000
5/5 [==============================] - 0s 4ms/step - loss: 17.4819 - val_loss: 28.8383
Epoch 1584/10000
5/5 [==============================] - 0s 4ms/step - loss: 16.7499 - val_loss: 22.0111
Epoch 1585/10000
5/5 [==============================] - 0s 4ms/step - loss: 14.6006 - val_loss: 22.6883
Epoch 1586/10000
5/5 [==============================] - 0s 4ms/step - loss: 14.0317 - val_loss: 31.2965
Epoch 1587/10000
5/5 [==============================] - 0s 4ms/step - loss: 14.4221 - val_loss: 33.7556
Epoch 1588/10000
5/5 [==============================] - 0s 4ms/step - loss: 12.6636 - val_loss: 19.7836
Epoch 1589/10000
5/5 [==============================] - 0s 4ms/step - loss: 15.5070 - val_loss: 20.5390
Epoch 1590/10000
5/5 [==============================] - 0s 4ms/step - loss: 15.0286 - val_loss: 31.7380
Epoch 1591/10000
5/5 [==============================] - 0s 4ms/s

Epoch 1661/10000
5/5 [==============================] - 0s 4ms/step - loss: 14.2085 - val_loss: 26.6595
Epoch 1662/10000
5/5 [==============================] - 0s 4ms/step - loss: 12.8864 - val_loss: 35.7633
Epoch 1663/10000
5/5 [==============================] - 0s 4ms/step - loss: 15.9559 - val_loss: 28.9872
Epoch 1664/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.7503 - val_loss: 23.3382
Epoch 1665/10000
5/5 [==============================] - 0s 4ms/step - loss: 14.8472 - val_loss: 39.8022
Epoch 1666/10000
5/5 [==============================] - 0s 4ms/step - loss: 14.6892 - val_loss: 35.7481
Epoch 1667/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.8515 - val_loss: 31.5780
Epoch 1668/10000
5/5 [==============================] - 0s 4ms/step - loss: 14.3783 - val_loss: 29.9803
Epoch 1669/10000
5/5 [==============================] - 0s 4ms/step - loss: 14.6413 - val_loss: 36.7599
Epoch 1670/10000
5/5 [==============================] - 0s 4ms/s

Epoch 1740/10000
5/5 [==============================] - 0s 4ms/step - loss: 17.6884 - val_loss: 41.2001
Epoch 1741/10000
5/5 [==============================] - 0s 4ms/step - loss: 16.3355 - val_loss: 22.7491
Epoch 1742/10000
5/5 [==============================] - 0s 5ms/step - loss: 17.7359 - val_loss: 25.5799
Epoch 1743/10000
5/5 [==============================] - 0s 5ms/step - loss: 14.1549 - val_loss: 21.5034
Epoch 1744/10000
5/5 [==============================] - 0s 5ms/step - loss: 15.1689 - val_loss: 33.1838
Epoch 1745/10000
5/5 [==============================] - 0s 5ms/step - loss: 13.3604 - val_loss: 31.9607
Epoch 1746/10000
5/5 [==============================] - 0s 5ms/step - loss: 13.4162 - val_loss: 23.6560
Epoch 1747/10000
5/5 [==============================] - 0s 5ms/step - loss: 13.8125 - val_loss: 28.9845
Epoch 1748/10000
5/5 [==============================] - 0s 5ms/step - loss: 15.9516 - val_loss: 36.2552
Epoch 1749/10000
5/5 [==============================] - 0s 4ms/s

In [23]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [24]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 13ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,53.553566,53.553543,53.553467,53.553482,53.552773,53.525467,11.442562,11.63927,21.789314,22.958199,4.008727,53.446518,3.484991,-8.091105,3.965674,12.857436,38.846252,-7.578867,-8.538363,-8.513203,-8.532068,-8.568899,-8.668713,-8.765547,-8.992937,-8.776243,-8.533916,-2.51832,48.971687,53.552322
Target,48.75,59.738,60.303,51.009,48.874,48.506,42.721,32.801,37.638,37.358,31.339,45.311,42.085,48.52,41.365,43.813,43.816,38.245,46.081,38.703,36.037,38.43,44.471,37.884,51.585,54.306,52.511,59.472,54.022,49.791
Error,4.803566,6.184456,6.749535,2.544483,4.678772,5.019466,31.278439,21.161728,15.848686,14.399803,27.330273,8.135517,38.60001,56.611107,37.399326,30.955563,4.969749,45.823868,54.619362,47.216202,44.569069,46.998901,53.139713,46.649544,60.577934,63.082245,61.044918,61.990318,5.050312,3.761322


In [25]:
display(mae)
display(mape)

30.373137

0.6935299

In [26]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [27]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[446.5769]] - Target[544.3480000000001]| =  Error: [[97.77112]]; MAPE:[[0.17961141]]
1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[-8.102407]] - Target[499.45000000000005]| =  Error: [[507.55243]]; MAPE:[[1.0162227]]
1/1 [==============================] - 0s 13ms/step
Ano-5: |Prediction[[73.70259]] - Target[321.687]| =  Error: [[247.98442]]; MAPE:[[0.77088726]]


[array([[97.77112]], dtype=float32),
 array([[507.55243]], dtype=float32),
 array([[247.98442]], dtype=float32)]

284.436

0.6555738